In [1]:
import math
from tensorboardX import SummaryWriter
import env_configurations
sess = None

/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-

In [2]:
import tensorflow as tf
import collections
from a2c_continuous import A2CAgent
import tr_helpers
import networks

gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers
import games_configurations
a2c_config = games_configurations.flexhum_config
env_name = a2c_config['ENV_NAME']#'RoboschoolHalfCheetah-v1'#'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#a2c_config = games_configurations.pendulum_lstm_config
#a2c_config = games_configurations.halfcheetah_lstm_config_v2
#a2c_config = games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = games_configurations.bipedalwalker_config
#a2c_config = games_configurations.roboschoolhumanoid_lstm_config

obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(obs_space)
print(action_space)
print(action_space.low)
print(action_space.high)


Box(76,)
Box(21,)
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-11-14 22:29:12,021	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-11-14_22-29-12_22901/logs.
2019-11-14 22:29:12,137	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:34166 to respond...
2019-11-14 22:29:12,258	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:62523 to respond...
2019-11-14 22:29:12,261	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-11-14 22:29:12,298	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-11-14_22-29-12_22901/logs.
2019-11-14 22:29:12,300	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.117',
 'redis_address': '192.168.3.117:34166',
 'object_store_address': '/tmp/ray/session_2019-11-14_22-29-12_22901/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-14_22-29-12_22901/sockets/raylet',
 'webui_url': None}

In [5]:




agent = A2CAgent(sess,'run4', obs_space, False, action_space, a2c_config)
#agent.restore('nn/latest_run_renderer_best')
agent.train()
#agent.save('nn/latest')

Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use `tf.cast` instead.


...

Frames per seconds:  9475.043402702615
saving next best rewards:  13.658103771228344
Frames per seconds:  9521.70976902369
saving next best rewards:  14.616724814213812
Frames per seconds:  9416.335113913927
saving next best rewards:  15.250439531356097
Frames per seconds:  9436.467078467045
saving next best rewards:  16.174792202189565
Frames per seconds:  9438.136692165419
Frames per seconds:  9508.938097787623
saving next best rewards:  16.546969420947136
Frames per seconds:  9619.133814480321
saving next best rewards:  16.564185988921672
Frames per seconds:  9664.281339610101
saving next best rewards:  17.915533297657966
Frames per seconds:  9420.822587334429
saving next best rewards:  19.580633482299746
Frames per seconds:  9939.043957210151
saving next best rewards:  20.59309150826186
Frames per seconds:  9850.344680260925
saving next best rewards:  22.27755602438003
Frames per seconds:  9921.902573379522
saving next best rewards:  22.817018445208667
Frames per seconds:  10006.23

Frames per seconds:  13570.091648874957
Frames per seconds:  13740.107886461097
Frames per seconds:  13531.424247100811
saving next best rewards:  1772.3985430637654
Frames per seconds:  13657.771488856992
saving next best rewards:  1883.6600263051316
Frames per seconds:  13580.152656543723
saving next best rewards:  1892.3369040917978
Frames per seconds:  13829.717039258825
saving next best rewards:  1913.9071697300672
Frames per seconds:  13723.905107021114
saving next best rewards:  1930.065892347619
Frames per seconds:  13391.573102551693
saving next best rewards:  2001.574046613574
Frames per seconds:  13438.320043889982
Frames per seconds:  13189.249170747473
saving next best rewards:  2036.2590676818788
Frames per seconds:  13134.069343616924
Frames per seconds:  12887.950492414135
saving next best rewards:  2075.2663033047693
Frames per seconds:  12835.124267857
Frames per seconds:  12758.92425704972
saving next best rewards:  2089.1401723587514
Frames per seconds:  13137.34856

Frames per seconds:  13814.742180260137
Frames per seconds:  13625.009644298194
saving next best rewards:  3817.465181979537
Frames per seconds:  13724.272382547433
Frames per seconds:  13770.413721424033
saving next best rewards:  3854.0339580404757
Frames per seconds:  13694.422875121336
Frames per seconds:  13814.560970934623
Frames per seconds:  13839.860942978614
Frames per seconds:  13621.710645962316
saving next best rewards:  3872.60722391963
Frames per seconds:  13514.802550466406
Frames per seconds:  13100.756294385119
Frames per seconds:  11932.393885035666
Frames per seconds:  12405.866787741532
saving next best rewards:  3905.400375780165
Frames per seconds:  13284.081479284205
saving next best rewards:  3917.345476137698
Frames per seconds:  13527.125860707338
Frames per seconds:  13303.838300990888
Frames per seconds:  12923.582574507456
Frames per seconds:  12575.56327034661
saving next best rewards:  3973.968627717495
Frames per seconds:  12926.593991204225
Frames per 

Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/ray/worker.py", line 1547, in print_logs
    msg = pubsub_client.get_message()
  File "/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/redis/client.py", line 3297, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/redis/client.py", line 3183, in parse_response
    if not block and not conn.can_read(timeout=timeout):
  File "/home/trrrrr/anaconda3/envs/tf14/lib/python3.6/site-packages/redis/connection.py", line 695, in can_read
    return self._parser.can_read(timeout)
  File "/home/trrrrr/anacon

KeyboardInterrupt: 

In [6]:
agent.save('nn/latest_lstm')
ray.shutdown()

In [ ]:
import gym
gym.envs.registry.all()